# Decompositions

This file demonstrates how the decomposition of quantum gates into the target basis is performed. Given its length, I have created a separate file for it. I will import the necessary functions from this file to complete part 2.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import copy
from helper_func import statevector_to_braket

Every unitary matrix can be decomposed into Z and Y rotations:
$$U = e^{i \alpha} R_{z}(\beta) R_{y}(\gamma) R_{z}(\delta) $$

The matrix form of $U_3$ gate is:
$$U_3(\theta, \phi, \lambda) = \begin{bmatrix}
\cos(\theta/2) & -e^{i\lambda}\sin(\theta/2) \\
e^{i\phi}\sin(\theta/2) & e^{i(\phi + \lambda)}\cos(\theta/2)
\end{bmatrix}$$

In order to write other unitaries in terms of $U_{3}$, the factor $e^{i \rho}$ is multiplied to it.

In [2]:
# Define supported gates
supported_gates = [
    'u', 'cx', 'rz', 'ry', 'rx', 'swap', 'cz', 'cy',
    'cu', 'crx', 'cs', 'ct', 'ssx', 'x', 'sx', 'id',
    'y', 'z', 'h', 's', 't'
]


# From comparision, we gather that
# alpha = beta/2 + delta/2 + rho
# beta = phi
# gamma = theta
# delta = lambda
def zy_decomposition(theta, phi, lam, rho):
    beta = phi
    delta = lam
    alpha = beta / 2 + delta / 2 + rho
    gamma = theta

    return alpha, beta, gamma, delta

```Gate``` class models quantum gates. It is used to decompose quantum gates into the target basis ```X```, ```CX```, ```SX```, ```RZ```, ```Id```. Decompositions are crucial for simplifying complex gates into elementary operations that can be executed on quantum hardware. It handles some single-qubit gates and controlled gates. 

```RY``` can be decomposed into target ```SX``` and ```RZ```:
$$R_{y}(\theta) = -i~SX~R_{z}(\pi - \theta)~SX~R_{z}(-\pi)$$

Building on this, if gates can be decomposed using $U$ gate, they can be simplified with $U_3$ gate and then further be decomposed to a series of  gates in the target basis.

In [3]:
class Gate:
    GATE_REQUIREMENTS = {
        'single_qubit': {'qubits': 1, 'params': 0},
        'one_control_two_qubits': {'qubits': 2, 'params': 0},
        'multi_control': {'qubits_min': 2, 'params': 0},
        'with_params': {
            'u': {'qubits': 1, 'params': 4},
            'cu': {'qubits': 2, 'params': 4},
            'rz': {'qubits': 1, 'params': 1},
            'ry': {'qubits': 1, 'params': 1},
            'rx': {'qubits': 1, 'params': 1},
            'crx': {'qubits': 2, 'params': 1},
            'ssx': {'qubits': 'variable', 'params': 1},
        }
    }

    # A map of the decompositions of the gates in terms of the gate U
    # Some gates can be directly decomposed to the target basis

    DECOMPOSITION_MAP = {
        'y': lambda q, p: [Gate('u', q, [np.pi, np.pi / 2, np.pi / 2, 0])],
        'z': lambda q, p: [Gate('u', q, [0, 0, np.pi, 0])],
        'h': lambda q, p: [Gate('u', q, [np.pi / 2, 0, np.pi, 0])],
        'ry': lambda q, p: Gate._decompose_ry_gate_static(q, p),
        'rx': lambda q, p: [Gate('u', q, [p[0], -np.pi / 2, np.pi / 2, 0])],
        's': lambda q, p: [Gate('u', q, [0, 0, np.pi / 2, 0])],
        't': lambda q, p: [Gate('u', q, [0, 0, np.pi / 4, 0])],
        'tdg': lambda q, p: [Gate('u', q, [0, 0, -np.pi / 4, 0])],
        'id': lambda q, p: [],
        'ssx': lambda q, p: Gate._decompose_ssx(q, p).decompose(),
        'cy': lambda q, p: [
            Gate('rz', q[0], [-np.pi / 2]),
            Gate('cx', q),
            Gate('rz', q[0], [np.pi / 2])
        ],
        'cz': lambda q, p: [
            Gate('u', [q[0]], [np.pi / 2, 0, np.pi, 0]),
            Gate('cx', q),
            Gate('u', [q[0]], [np.pi / 2, 0, np.pi, 0])
        ],
        'swap': lambda q, p: [
            Gate('cx', q),
            Gate('cx', [q[1], q[0]]),
            Gate('cx', q)
        ],
        'crx': lambda q, p: Gate._decompose_cu_gate_static(q, p, 'crx'),
        'cu': lambda q, p: Gate._decompose_cu_gate_static(q, p, 'cu'),
        'cs': lambda q, p: Gate._decompose_cu_gate_static(q, p, 'cs'),
        'ct': lambda q, p: Gate._decompose_cu_gate_static(q, p, 'ct'),
    }

    def __init__(self, name, qubits, params=None):
        self.name = name
        self.qubits = self._validate_qubits(qubits)
        self.params = params if params is not None else []

    def _validate_qubits(self, qubits):
        if isinstance(qubits, list):
            return qubits
        return [qubits]

    def target(self):
        return self.qubits[0]

    def control(self):
        return self.qubits[1:]

    def nr_qubits(self):
        return len(self.qubits)

    def decompose(self):
        gates = [self]
        for stage in ['_decompose_multiple_qubit_gates', '_decompose_to_cx_rz_u', '_decompose_u_gate']:
            decomposed = []
            for gate in gates:
                decomposed.extend(getattr(gate, stage)())
            gates = decomposed
        return gates

    def _decompose_multiple_qubit_gates(self):
        if self.nr_qubits() <= 2:
            return [self]
        
        if self.name not in ['cx', 'ssx']:
            return [self]
        
        control_qubits = self.control().copy()
        target_qubit = self.target()
        first_control = control_qubits.pop(0)
        k = self.params[0] if self.name == 'ssx' else 0
        k_succ = k + 1 if k >= 0 else k - 1

        decomposition = [
            Gate('ssx', [target_qubit, first_control], [k_succ]),
            *Gate('cx', [first_control] + control_qubits)._decompose_multiple_qubit_gates(),
            Gate('ssx', [target_qubit, first_control], [-k_succ]),
            copy.deepcopy(Gate('cx', [first_control] + control_qubits)._decompose_multiple_qubit_gates()),
            Gate('ssx', [target_qubit] + control_qubits, [k_succ])
        ]
        return decomposition

    def _decompose_to_cx_rz_u(self):
        decomposition_func = self.DECOMPOSITION_MAP.get(self.name, lambda q, p: [self])
        return decomposition_func(self.qubits, self.params)

    def _decompose_u_gate(self):
        if self.name != 'u':
            return [self]
        
        _, beta, gamma, delta = zy_decomposition(*self.params)
        gates = []
        if delta != 0:
            gates.append(Gate('rz', self.qubits, [delta]))
        gates += Gate('ry', self.qubits, [gamma])._decompose_ry_gate()
        if beta != 0:
            gates.append(Gate('rz', self.qubits, [beta]))
        return gates

    @staticmethod
    def _decompose_ry_gate_static(qubits, params):
        return [
            Gate('rz', qubits, [-np.pi]),
            Gate('sx', qubits),
            Gate('rz', qubits, [np.pi - params[0]]),
            Gate('sx', qubits)
        ]

    def _decompose_ry_gate(self):
        if self.name != 'ry':
            return [self]
        return [
            Gate('rz', self.qubits, [-np.pi]),
            Gate('sx', self.qubits),
            Gate('rz', self.qubits, [np.pi - self.params[0]]),
            Gate('sx', self.qubits)
        ]

    @staticmethod
    def _decompose_ssx(qubits, params):
        return Gate('ssx', qubits, params)

    @staticmethod
    def _decompose_cu_gate_static(qubits, params, gate_type):
        if gate_type == 'crx':
            theta, phi, lam, rho = params[0], -np.pi / 2, np.pi / 2, 0
        elif gate_type == 'cs':
            theta, phi, lam, rho = 0, 0, np.pi / 2, 0
        elif gate_type == 'ct':
            theta, phi, lam, rho = 0, 0, np.pi / 4, 0
        elif gate_type == 'cu':
            theta, phi, lam, rho = params
        else:
            return [Gate(gate_type, qubits, params)]

        alpha, beta, gamma, delta = zy_decomposition(theta, phi, lam, rho)
        phase_shift = [Gate('u', [qubits[0]], [0, 0, alpha, 0])] if alpha != 0 else []
        A = Gate('ry', [qubits[1]], [gamma / 2])._decompose_ry_gate() + [Gate('rz', [qubits[1]], [beta])]
        B = [Gate('rz', [qubits[1]], [-(beta + delta) / 2])] + Gate('ry', [qubits[1]], [-gamma / 2])._decompose_ry_gate()
        C = [Gate('rz', [qubits[1]], [(delta - beta) / 2])]
        CX = [Gate('cx', qubits)]
        
        return C + CX + B + copy.deepcopy(CX) + A + phase_shift

    def _decompose_cu_gate(self):
        return self._decompose_cu_gate_static(self.qubits, self.params, self.name)


The input is given as a list of operations as seen in the previous file. First convert the list of pennylane operations in order to use the ```Gate``` class.

In [4]:
def decompose_pennylane_ops(ops):
    decomposed = []
    for op in ops:
        name = op.name.lower()
        params = list(op.parameters)
        wires = [int(wire) for wire in op.wires]

        if name in ['pauli_x', 'x']:
            name = 'x'
        elif name in ['pauli_y', 'y']:
            name = 'y'
        elif name in ['pauli_z', 'z']:
            name = 'z'
        elif name in ['hadamard', 'h']:
            name = 'h'
        elif name == 's':
            name = 's'
        elif name == 't':
            name = 't'
        elif name in ['cnot', 'cx']:
            name = 'cx'
        elif name == 'crx':
            name = 'crx'
        elif name == 'cu3':
            name = 'cu'
            params = params + [0]
        elif name == 'swap':
            name = 'swap'
        elif name in ['cy', 'cz', 'cs', 'ct', 'ssx']:
            name = name 
        elif name in ['rx', 'ry', 'rz', 'u', 'id']:
            name = name

        gate = Gate(name, wires, params)
        decomposed_gates = gate.decompose()
        decomposed.extend(decomposed_gates)
        
    return decomposed

In [5]:
# A list of Gate objects where all the quantum gates have been decomposed to the target basis
def get_decomposed_gates(gates):
    decomposed_gates = []
    for gate in gates:
        decomposed_gates.append(Gate(name=gate.name, qubits=gate.qubits, params=gate.params))
    
    return decomposed_gates


In [6]:
# Mapping from custom gate names to PennyLane gate functions
GATE_MAPPING = {
    'cx': qml.CNOT,
    'cz': qml.CZ,
    'cy': qml.CY,
    'swap': qml.SWAP,
    'crx': qml.CRX,
    'cs': qml.CSWAP,
    'ct': qml.CZ,
    'rz': qml.RZ,
    'ry': qml.RY,
    'rx': qml.RX,
    'u': qml.U3,
    'sx': qml.SX,
    'x': qml.PauliX,
    'y': qml.PauliY,
    'z': qml.PauliZ,
    'h': qml.Hadamard,
    's': qml.S,
    't': qml.T,
    'id': qml.Identity,
    # Add more mappings as needed
}


Convert the objects of Gate class into a pennylane quantum circuit.

In [7]:
def convert_gates_to_pennylane_circuit(gates, return_type='state'):
    all_qubits = set()
    for gate in gates:
        all_qubits.update(gate.qubits)
    num_wires = max(all_qubits) + 1

    dev = qml.device('default.qubit', wires=num_wires)

    def circuit():
        for gate in gates:
            gate_name = gate.name.lower()
            qubits = gate.qubits
            params = gate.params

            qml_gate = GATE_MAPPING.get(gate_name)

            if qml_gate.num_params == 0:
                qml_gate(wires=qubits)
            elif qml_gate.num_params == 1:
                qml_gate(params[0], wires=qubits)
            elif qml_gate.num_params == 2:
                qml_gate(params[0], params[1], wires=qubits)
            elif qml_gate.num_params == 3:
                qml_gate(params[0], params[1], params[2], wires=qubits)
            elif qml_gate.num_params == 4:
                qml_gate(params[0], params[1], params[2], params[3], wires=qubits)

        if return_type == 'state':
            return qml.state()
        elif return_type == 'probabilities':
            return qml.probs(wires=range(num_wires))
        elif return_type == 'expectation':
            return [qml.expval(qml.PauliZ(wire)) for wire in range(num_wires)]


    qnode = qml.QNode(circuit, dev)

    return qnode()


Example with a list of Pennylane operations to compare the final statevector with the original operations and the statevector with the decomposed operations.

In [8]:
ops = [
    qml.RX(0.2, wires=0),
    qml.CNOT(wires=[0, 1]),
    qml.Hadamard(wires=2),
    qml.RY(np.pi / 4, wires=1),
    qml.SWAP(wires=[0, 1])
]

In [9]:
decomposed_gates = decompose_pennylane_ops(ops)
decomposed_gates_list = get_decomposed_gates(decomposed_gates)
statevector_with_decomposition = convert_gates_to_pennylane_circuit(decomposed_gates_list, return_type='state')

In [10]:
dev2 = qml.device('default.qubit', wires=3)
@qml.qnode(dev2)
def circuit2():
    qml.RX(0.2, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=2)
    qml.RY(np.pi / 4, wires=1)
    qml.SWAP(wires=[0, 1])
    return qml.state()

statevector_original = circuit2().numpy()
print(f'State without decompositions: {statevector_to_braket(statevector_original)}')
print(f'State with decompositions: {statevector_to_braket(statevector_with_decomposition)}')

State without decompositions: 0.65+0.00j * |000> + 0.65+0.00j * |001> + 0.00+0.03j * |010> + 0.00+0.03j * |011> + 0.27+0.00j * |100> + 0.27+0.00j * |101> + 0.00-0.07j * |110> + 0.00-0.07j * |111>
State with decompositions: -0.65+0.00j * |000> + -0.65+0.00j * |001> + 0.00-0.03j * |010> + 0.00-0.03j * |011> + -0.27+0.00j * |100> + -0.27+0.00j * |101> + 0.00+0.07j * |110> + 0.00+0.07j * |111>


The statevectors differ by a factor of $-1$. Since it is a global phase, it can be safely ignored.

### References

- M. Nielsen and I. L. Chuang, “Quantum computation and quantum information,” Cambridge University Press, Cambridge
- Elementary gates for quantum computation, Phys. Rev. A 52, 3457

Proceed to Part_2.ipynb